In [55]:
import pandas as pd
import string
import re
import ast
from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [56]:
# Read ROCStories into pandas DataFrame
roc_stories_path = '../generated/knowledge-triple-extraction/ROCStories_resolved_with_knowledge_triples.csv'
roc_stories_df = pd.read_csv(roc_stories_path, sep='\t', header=0)
del roc_stories_df['srl_r1']
del roc_stories_df['Unnamed: 0']
del roc_stories_df['Unnamed: 0.1']
del roc_stories_df['Unnamed: 0.1.1']
del roc_stories_df['Unnamed: 0.1.1.1']

In [57]:
roc_stories_df.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5,triples1,triples2,triples3,triples4,triples5
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a change.,They got themselves and Dan on a diet.,"[Dan: [Dan, Dan, his, His, Dan], Dan's parents: [Dan's parents, his parents, His parents, They, themselves]]",Dan's parents were overweight.,Dan was overweight as well.,The doctors told Dan's parents it was unhealthy.,Dan's parents understood and decided to make a change.,Dan's parents got Dan's parents and Dan on a diet.,"[[""Dan 's parents"", 'were', 'overweight']]","[['Dan', 'was', 'overweight'], ['Dan', 'was', 'as well']]","[['The doctors', 'told', ""Dan 's parents""], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]","[[""Dan 's parents"", 'decided', 'to make a change'], [""Dan 's parents"", 'make', 'a change']]","[[""Dan 's parents"", 'got', ""Dan 's parents and Dan""], [""Dan 's parents"", 'got', 'on a diet']]"
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a wall.,The bike frame bent and Carrie got a deep gash on her leg.,"[Carrie: [Carrie, She, her, Carrie, her, She, Carrie, her]]",Carrie had just learned how to ride a bike.,Carrie didn't have a bike of Carrie own.,Carrie would sneak rides on Carrie sister's bike.,Carrie got nervous on a hill and crashed into a wall.,The bike frame bent and Carrie got a deep gash on Carrie leg.,"[['Carrie', 'learned', 'just'], ['Carrie', 'learned', 'how to ride a bike'], ['Carrie', 'ride', 'how'], ['Carrie', 'ride', 'a bike']]","[['Carrie', 'have', ""n't""], ['Carrie', 'have', 'a bike of Carrie own']]","[['Carrie', 'sneak', 'would'], ['Carrie', 'sneak', ""rides on Carrie sister 's bike""]]","[['Carrie', 'got', 'nervous'], ['Carrie', 'got', 'on a hill'], ['Carrie', 'crashed', 'into a wall']]","[['Carrie', 'got', 'a deep gash on Carrie leg']]"
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to her first.,"[Morgan: [Morgan, She, her, Morgan, her, Her, he], her boyfriend: [her boyfriend, her boyfriend, Her boyfriend, her]]",Morgan enjoyed long walks on the beach.,Morgan and Morgan boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,her boyfriend was upset Morgan didn't propose to her boyfriend first.,"[['Morgan', 'enjoyed', 'long walks on the beach']]","[['Morgan and Morgan boyfriend', 'decided', 'to go for a long walk'], ['Morgan and Morgan boyfriend', 'go', 'for a long walk']]","[['something', 'happened', 'After walking for over a mile']]","[['Morgan', 'decided', 'to propose to her boyfriend'], ['Morgan', 'propose', 'to her boyfriend']]","[['her boyfriend', 'was', ""upset Morgan did n't propose to her boyfriend first""], [""Morgan did n't propose to her boyfriend first"", 'upset', 'her boyfriend'], ['Morgan', 'propose', ""n't""], ['Morgan', 'propose', 'to her boyfriend'], ['Morgan', 'propose', 'first']]"
3,d173b7de-4611-4cdf-934c-912834755e41,The bad customer.,Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",He began yelling about how long his food was taking.,Jane didn't know how to react.,"Luckily, her coworker intervened and calmed the man down.","[Jane: [Jane, Jane, her], a customer: [a customer, He, his, the man]]",Jane was working at a diner.,"Suddenly, a customer barged up to the counter.",a customer began yelling abou

In [58]:
punctuation = set(['' if c == ' ' else c for c in string.punctuation])

In [59]:
def remove_punctuation(text):
    return ''.join(['' if c in punctuation else c for c in text])

In [100]:
def extract_bigrams(text):
    res = []
    text = remove_punctuation(text)
    words = text.lower().split()
    two_grams = [" ".join(words[i:i+2]) for i in range(len(words) - 1)]
    two_grams = []
    two_grams.append(text.lower())
    return list(set(two_grams))

In [101]:
def extract_bigrams_from_triple(triple):
    new_triple = triple
    new_triple[0] = extract_bigrams(triple[0])
    new_triple[1] = triple[1].split()
    new_triple[2] = extract_bigrams(triple[2])
    return new_triple

In [102]:
def extract_bigrams_from_triples(triples):
    triples = ast.literal_eval(triples)
    new_triples = [[], [], []]
    for triple in triples:
        bigram_triple = extract_bigrams_from_triple(triple)
        new_triples[0] += bigram_triple[0]
        new_triples[1] += bigram_triple[1]
        new_triples[2] += bigram_triple[2]
    return [set(l) for l in new_triples]

In [103]:
for n in range(1, 6):
    roc_stories_df[f'bigram_triples_{n}'] = roc_stories_df[f'triples{n}'].progress_apply(extract_bigrams_from_triples)

100%|██████████| 98161/98161 [00:04<00:00, 22827.12it/s]


In [104]:
roc_stories_df.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5,triples1,triples2,triples3,triples4,triples5,bigram_triples_1,bigram_triples_2,bigram_triples_3,bigram_triples_4,bigram_triples_5,tail_tail_overlap_1_3,tail_head_overlap_1_3,verb_verb_overlap_1_3,tail_tail_overlap_1_4,tail_head_overlap_1_4,verb_verb_overlap_1_4,tail_tail_overlap_2_4,tail_head_overlap_2_4,verb_verb_overlap_2_4,tail_tail_overlap_3_5,tail_head_overlap_3_5,verb_verb_overlap_3_5,has_overlap
0,9a51198e-96f1-42c3-b09d-a3e1e067d803,Overweight Kid,Dan's parents were overweight.,Dan was overweight as well.,The doctors told his parents it was unhealthy.,His parents understood and decided to make a change.,They got themselves and Dan on a diet.,"[Dan: [Dan, Dan, his, His, Dan], Dan's parents: [Dan's parents, his parents, His parents, They, themselves]]",Dan's parents were overweight.,Dan was overweight as well.,The doctors told Dan's parents it was unhealthy.,Dan's parents understood and decided to make a change.,Dan's parents got Dan's parents and Dan on a diet.,"[[""Dan 's parents"", 'were', 'overweight']]","[['Dan', 'was', 'overweight'], ['Dan', 'was', 'as well']]","[['The doctors', 'told', ""Dan 's parents""], ['The doctors', 'told', 'it was unhealthy'], ['it', 'was', 'unhealthy']]","[[""Dan 's parents"", 'decided', 'to make a change'], [""Dan 's parents"", 'make', 'a change']]","[[""Dan 's parents"", 'got', ""Dan 's parents and Dan""], [""Dan 's parents"", 'got', 'on a diet']]","[{dan s parents}, {were}, {overweight}]","[{dan}, {was}, {overweight, as well}]","[{the doctors, it}, {was, told}, {it was unhealthy, unhealthy, dan s parents}]","[{dan s parents}, {make, decided}, {a change, to make a change}]","[{dan s parents}, {got}, {on a diet, dan s parents and dan}]",False,True,False,False,False,False,False,False,False,True,False,False,True
1,617e7ada-3878-488d-bd56-40695b91f053,The Bike Accident,Carrie had just learned how to ride a bike.,She didn't have a bike of her own.,Carrie would sneak rides on her sister's bike.,She got nervous on a hill and crashed into a wall.,The bike frame bent and Carrie got a deep gash on her leg.,"[Carrie: [Carrie, She, her, Carrie, her, She, Carrie, her]]",Carrie had just learned how to ride a bike.,Carrie didn't have a bike of Carrie own.,Carrie would sneak rides on Carrie sister's bike.,Carrie got nervous on a hill and crashed into a wall.,The bike frame bent and Carrie got a deep gash on Carrie leg.,"[['Carrie', 'learned', 'just'], ['Carrie', 'learned', 'how to ride a bike'], ['Carrie', 'ride', 'how'], ['Carrie', 'ride', 'a bike']]","[['Carrie', 'have', ""n't""], ['Carrie', 'have', 'a bike of Carrie own']]","[['Carrie', 'sneak', 'would'], ['Carrie', 'sneak', ""rides on Carrie sister 's bike""]]","[['Carrie', 'got', 'nervous'], ['Carrie', 'got', 'on a hill'], ['Carrie', 'crashed', 'into a wall']]","[['Carrie', 'got', 'a deep gash on Carrie leg']]","[{carrie}, {ride, learned}, {just, how, how to ride a bike, a bike}]","[{carrie}, {have}, {a bike of carrie own, nt}]","[{carrie}, {sneak}, {would, rides on carrie sister s bike}]","[{carrie}, {got, crashed}, {into a wall, on a hill, nervous}]","[{carrie}, {got}, {a deep gash on carrie leg}]",False,False,False,False,False,False,False,False,False,True,False,False,True
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to her first.,"[Morgan: [Morgan, She, her, Morgan, her, Her, he], her boyfriend: [her boyfriend, her boyfriend, Her boyfriend, her]]",Morgan enjoyed long walks on the beach.,Morgan and Morgan boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,her boyfriend was upset Morgan didn't propose 

In [105]:
def bigram_overlap(set1, set2):
    return bool(set1 & set2)

In [106]:
overlapping_sentences = [(1,3), (1,4), (2,4), (3,5)]

In [115]:
for src, tgt in overlapping_sentences:
    roc_stories_df[f'tail_tail_overlap_{src}_{tgt}'] = roc_stories_df[[f'bigram_triples_{src}', f'bigram_triples_{tgt}']].progress_apply(lambda x: bigram_overlap(x[f'bigram_triples_{src}'][2], x[f'bigram_triples_{tgt}'][2]), axis=1)
    roc_stories_df[f'tail_head_overlap_{src}_{tgt}'] = roc_stories_df[[f'bigram_triples_{src}', f'bigram_triples_{tgt}']].progress_apply(lambda x: bigram_overlap(x[f'bigram_triples_{src}'][2], x[f'bigram_triples_{tgt}'][0]), axis=1)
    roc_stories_df[f'verb_verb_overlap_{src}_{tgt}'] = roc_stories_df[[f'bigram_triples_{src}', f'bigram_triples_{tgt}']].progress_apply(lambda x: bigram_overlap(x[f'bigram_triples_{src}'][1], x[f'bigram_triples_{tgt}'][1]), axis=1)

100%|██████████| 98161/98161 [00:01<00:00, 89835.26it/s]


In [137]:
roc_stories_df['has_overlap'] = False

In [138]:
has_overlap = roc_stories_df['has_overlap']
for src, tgt in overlapping_sentences:
    has_overlap |= roc_stories_df[f'verb_verb_overlap_{src}_{tgt}']
pruned_ds = roc_stories_df[has_overlap]

In [139]:
len(pruned_ds)

21771

In [140]:
pruned_ds.head()

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,coref_clusters,resolved1,resolved2,resolved3,resolved4,resolved5,triples1,triples2,triples3,triples4,triples5,bigram_triples_1,bigram_triples_2,bigram_triples_3,bigram_triples_4,bigram_triples_5,tail_tail_overlap_1_3,tail_head_overlap_1_3,verb_verb_overlap_1_3,tail_tail_overlap_1_4,tail_head_overlap_1_4,verb_verb_overlap_1_4,tail_tail_overlap_2_4,tail_head_overlap_2_4,verb_verb_overlap_2_4,tail_tail_overlap_3_5,tail_head_overlap_3_5,verb_verb_overlap_3_5,has_overlap
2,79b0da1f-e460-4173-ba58-8c9e2553c53a,Beach,Morgan enjoyed long walks on the beach.,She and her boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,Her boyfriend was upset he didn't propose to her first.,"[Morgan: [Morgan, She, her, Morgan, her, Her, he], her boyfriend: [her boyfriend, her boyfriend, Her boyfriend, her]]",Morgan enjoyed long walks on the beach.,Morgan and Morgan boyfriend decided to go for a long walk.,"After walking for over a mile, something happened.",Morgan decided to propose to her boyfriend.,her boyfriend was upset Morgan didn't propose to her boyfriend first.,"[['Morgan', 'enjoyed', 'long walks on the beach']]","[['Morgan and Morgan boyfriend', 'decided', 'to go for a long walk'], ['Morgan and Morgan boyfriend', 'go', 'for a long walk']]","[['something', 'happened', 'After walking for over a mile']]","[['Morgan', 'decided', 'to propose to her boyfriend'], ['Morgan', 'propose', 'to her boyfriend']]","[['her boyfriend', 'was', ""upset Morgan did n't propose to her boyfriend first""], [""Morgan did n't propose to her boyfriend first"", 'upset', 'her boyfriend'], ['Morgan', 'propose', ""n't""], ['Morgan', 'propose', 'to her boyfriend'], ['Morgan', 'propose', 'first']]","[{morgan}, {enjoyed}, {long walks on the beach}]","[{morgan and morgan boyfriend}, {decided, go}, {to go for a long walk, for a long walk}]","[{something}, {happened}, {after walking for over a mile}]","[{morgan}, {propose, decided}, {to propose to her boyfriend, to her boyfriend}]","[{her boyfriend, morgan did nt propose to her boyfriend first, morgan}, {was, upset, propose}, {first, upset morgan did nt propose to her boyfriend first, to her boyfriend, nt, her boyfriend}]",False,False,False,False,False,False,False,False,True,False,False,False,True
5,5d5e7aeb-332f-4c8b-a3c3-44585501e493,Foolish Frank,Frank had been drinking beer.,"He got a call from his girlfriend, asking where he was.",Frank suddenly realized he had a date that night.,"Since Frank was already a bit drunk, he could not drive.",Frank spent the rest of the night drinking more beers.,"[Frank: [Frank, He, his, he, Frank, he, Frank, he, Frank]]",Frank had been drinking beer.,"Frank got a call from Frank girlfriend, asking where Frank was.",Frank suddenly realized Frank had a date that night.,"Since Frank was already a bit drunk, Frank could not drive.",Frank spent the rest of the night drinking more beers.,"[['Frank', 'drinking', 'beer']]","[['Frank', 'got', 'a call'], ['Frank', 'got', 'from Frank girlfriend'], ['Frank', 'got', 'asking where Frank was'], ['Frank', 'asking', 'where Frank was'], ['Frank', 'was', 'where']]","[['Frank', 'realized', 'suddenly'], ['Frank', 'realized', 'Frank had a date that night'], ['Frank', 'had', 'a date'], ['Frank', 'had', 'that night']]","[['Frank', 'was', 'already'], ['Frank', 'was', 'a bit drunk'], ['Frank', 'drive', 'Since Frank was already a bit drunk'], ['Frank', 'drive', 'could'], ['Frank', 'drive', 'not']]","[['Frank', 'spent', 'the rest of the night'], ['Frank', 'spent', 'drinking more beers'], ['Frank', 'drinking', 'more beers']]","[{frank}, {drinking}, {beer}]","[{frank}, {was, got, asking}, {from frank girlfriend, a call, where frank was, where, asking where frank was}]","[{frank}, {had, realized}, {frank had a date that night, that night, a date, suddenly}]","[{frank}, {was, drive}, {since frank was already a bit drunk,